In [234]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
to_run = True
if to_run:
    os.chdir('..')
    to_run = False
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\ricca\Documents\Copenaghen-Classes\REBS
c:\Users\ricca\Documents\Copenaghen-Classes


In [235]:
############################################################################################
#EXAMPLES 1
############################################################################################

from pm4py.objects.dcr.multi_instance.obj import MultiInstanceDcrGraph
from pm4py.objects.dcr.multi_instance.semantics import MultiInstanceSemantics

graph = MultiInstanceDcrGraph()

events = ["A", "B"]
for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    graph.marking.included.add(event)

graph.marking.included.discard("B") #B is 'abstract, not instantiated'
graph.spawn["A"] = {"P"}
graph.subprocesses["P"] = {"B"}
graph.excludes["B"] = {"B"}

print("Before execution")
print("A should be enabled, expected True:")
print(MultiInstanceSemantics.is_enabled("A", graph))

print("B should not be enabled, expected False:")
print(MultiInstanceSemantics.is_enabled("B", graph))

#We execute A twice
MultiInstanceSemantics.execute(graph, "A")
MultiInstanceSemantics.execute(graph, "A")

print("After execution:")
print("A, B:P1 and B:P2 should all be enabled, expected: True, True, True:")
print(f"{MultiInstanceSemantics.is_enabled('A', graph)} {MultiInstanceSemantics.is_enabled('B:P1', graph)} {MultiInstanceSemantics.is_enabled('B:P2', graph)}")

MultiInstanceSemantics.execute(graph, "B:P1")
MultiInstanceSemantics.execute(graph, "B:P2")
print("After execution of B:P1 and B:P2 :")
print("B:P1 should not be enabled, expected False:")
print(MultiInstanceSemantics.is_enabled("B:P1", graph))
print("B:P2 should not be enabled, expected False:")
print(MultiInstanceSemantics.is_enabled("B:P2", graph))


Before execution
A should be enabled, expected True:
True
B should not be enabled, expected False:
False
After execution:
A, B:P1 and B:P2 should all be enabled, expected: True, True, True:
True True True
After execution of B:P1 and B:P2 :
B:P1 should not be enabled, expected False:
False
B:P2 should not be enabled, expected False:
False


In [236]:
############################################################################################
#EXAMPLES 2
############################################################################################

from pm4py.objects.dcr.multi_instance.obj import MultiInstanceDcrGraph
from pm4py.objects.dcr.multi_instance.semantics import MultiInstanceSemantics

graph = MultiInstanceDcrGraph()

events = ["A", "B"]
for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    graph.marking.included.add(event)

graph.spawn["A"] = {"P"}
graph.subprocesses["P"] = {"B"}

graph.marking.included.discard("B")

graph.excludes["B"] = {"B"}
graph.responses["A"] = {"B"}

print("Before executin, A should be enabled, expected True:")
print(MultiInstanceSemantics.is_enabled("A", graph))

#We execute A twice
MultiInstanceSemantics.execute(graph, "A")
MultiInstanceSemantics.execute(graph, "A")

print("After 2 executions of A:")
print("B:P1 should be enabled, expected True:")
print(MultiInstanceSemantics.is_enabled("B:P1", graph))
print("B:P2 should be enabled, expected True:")
print(MultiInstanceSemantics.is_enabled("B:P2", graph))
print("These are the events that are pending:")
print(graph.marking.pending)


Before executin, A should be enabled, expected True:
True
After 2 executions of A:
B:P1 should be enabled, expected True:
True
B:P2 should be enabled, expected True:
True
These are the events that are pending:
{'B', 'B:P1', 'B:P2'}


In [237]:
############################################################################################
#EXAMPLES 4
############################################################################################

from pm4py.objects.dcr.multi_instance.obj import MultiInstanceDcrGraph
from pm4py.objects.dcr.multi_instance.semantics import MultiInstanceSemantics

graph = MultiInstanceDcrGraph()

events = ["A", "B", "C"]
for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    graph.marking.included.add(event)

graph.marking.included.discard("B")
graph.marking.included.discard("C")

graph.spawn["A"] = {"P"}
graph.subprocesses["P"] = {"B", "C"}

graph.excludes["B"] = {"B"}
graph.excludes["C"] = {"C"}

graph.conditions["C"] = {"B"}

#We execute A twice
MultiInstanceSemantics.execute(graph, "A")
MultiInstanceSemantics.execute(graph, "A")

print("After 2 executions of A:")
print("B:P1 should be enabled, expected True")
print(MultiInstanceSemantics.is_enabled("B:P1", graph))
print("B:P2 should be enabled, expected True")
print(MultiInstanceSemantics.is_enabled("B:P2", graph))
print("C:P1 should not be enabled, expected False")
print(MultiInstanceSemantics.is_enabled("C:P1", graph))
print("C:P2 should not be enabled, expected False")
print(MultiInstanceSemantics.is_enabled("C:P2", graph))
print("\nThose values are due to the condition B->C")

After 2 executions of A:
B:P1 should be enabled, expected True
True
B:P2 should be enabled, expected True
True
C:P1 should not be enabled, expected False
False
C:P2 should not be enabled, expected False
False

Those values are due to the condition B->C


In [238]:
############################################################################################
#EXAMPLES 5
############################################################################################

from pm4py.objects.dcr.multi_instance.obj import MultiInstanceDcrGraph
from pm4py.objects.dcr.multi_instance.semantics import MultiInstanceSemantics

graph = MultiInstanceDcrGraph()

events = ["A", "B", "C"]
for event in events:
    graph.events.add(event)
    graph.labels.add(event)
    graph.label_map[event] = event
    graph.marking.included.add(event)

graph.marking.included.discard("B")
graph.marking.included.discard("C")

graph.spawn["A"] = {"P"}
graph.subprocesses["P"] = {"B", "C"}

graph.excludes["A"] = {"A"}
graph.excludes["B"] = {"B"}
graph.excludes["C"] = {"C"}

graph.conditions["C"] = {"B"}

graph.includes["C"] = {"A"}

print("State before execution:")
for event in graph.events:
    print(f"{event} is enabled: {MultiInstanceSemantics.is_enabled(event, graph)}")

#We execute A twice
MultiInstanceSemantics.execute(graph, "A")

print("\nAfter one execution of A:")
print("A should not be enabled, expected False:")
print(MultiInstanceSemantics.is_enabled("A", graph))
print("B:P1 should be enabled, expected True:")
print(MultiInstanceSemantics.is_enabled("B:P1", graph))
print("C:P1 should not be enabled, expected False:")
print(MultiInstanceSemantics.is_enabled("C:P1", graph))

print("After execution of B:P1, and subsequent execution of C:P1 :")
MultiInstanceSemantics.execute(graph, "B:P1")
MultiInstanceSemantics.execute(graph, "C:P1")
print("C:P1 has been executed so A should be enabled, expected True:")
print(MultiInstanceSemantics.is_enabled("A", graph))




State before execution:
B is enabled: False
A is enabled: True
C is enabled: False

After one execution of A:
A should not be enabled, expected False:
False
B:P1 should be enabled, expected True:
True
C:P1 should not be enabled, expected False:
False
After execution of B:P1, and subsequent execution of C:P1 :
C:P1 has been executed so A should be enabled, expected True:
True
